In [1]:
from jetstream_hugo.definitions import *
from jetstream_hugo.plots import *
from jetstream_hugo.data import *
from jetstream_hugo.anyspell import *
from jetstream_hugo.jet_finding import *
from jetstream_hugo.clustering import *
import geopandas as gpd

import intake

import colormaps

%load_ext autoreload
%autoreload 2
%matplotlib inline

# arco-era5 tests

In [3]:
from dask.diagnostics import ProgressBar
def _compute(obj, progress: bool = False, **kwargs):
    kwargs = COMPUTE_KWARGS | kwargs
    try:
        if progress:
            with ProgressBar():
                return obj.compute(**kwargs)
        else:
            return obj.compute(**kwargs)
    except AttributeError:
        return obj

In [2]:
import xarray as xr

ds = xr.open_zarr(
    'gs://gcp-public-data-arco-era5/ar/full_37-1h-0p25deg-chunk-1.zarr-v3',
    chunks=None,
    storage_options=dict(token='anon'),
)
ar_full_37_1h = ds.sel(time=slice(ds.attrs['valid_time_start'], ds.attrs['valid_time_stop']))


In [16]:
base_ds = ar_full_37_1h[["u_component_of_wind", "v_component_of_wind"]].sel(time=ar_full_37_1h.time.dt.hour % 6 == 0, latitude=ar_full_37_1h.latitude >= 0, level=[175,  200,  225,  250,  300,  350]).isel(longitude=slice(None, None, 2), latitude=slice(None, None, 2))

In [ ]:
base_path = Path(f"/storage/workspaces/giub_meteo_impacts/ci01/ERA5/plev/flat_wind/dailymean")
from concurrent.futures import ThreadPoolExecutor, as_completed

def downloader(base_ds, base_path, month, year):
    month_str = str(month).zfill(2)
    opath = base_path.joinpath(f"{year}{month_str}.nc")
    if opath.is_file():
        return f"Already had {year}{month}"
    ds = _compute(base_ds.sel(time=(base_ds.time.dt.year==year) & (base_ds.time.dt.month==month)), progress=True)
    ds = standardize(ds)
    ds["s"] = np.sqrt(ds["u"] ** 2 + ds["v"] ** 2)
    ds = flatten_by(ds, "s")
    ds.to_netcdf(opath)
    return f"Completed {year}{month}"

with ThreadPoolExecutor(max_workers=6) as executor:
    futures = [
        executor.submit(downloader, base_ds.copy(), base_path, month, year) for year in YEARS for month in range(1, 13)
    ]
    for f in as_completed(futures):
        try:
            print(f.result())
        except:
            print("could not retrieve")

In [ ]:
varnames = {
    "t2m": "2m_temperature",
    "mslp": "mean_sea_level_pressure",
    "sst": "sea_surface_temperature",
    "tp": "total_precipitation",
}
for key, val in varnames.items():
    base_da = ar_full_37_1h[val].sel(time=ar_full_37_1h.time.dt.hour % 6 == 0, latitude=ar_full_37_1h.latitude >= 0)[:, ::2, ::2]
    base_path = Path(f"/storage/workspaces/giub_meteo_impacts/ci01/ERA5/surf/{key}/dailymean")
    print(base_path)
    for year in tqdm(YEARS):
        opath = base_path.joinpath(f"{year}.nc")
        if opath.is_file():
            da = xr.open_dataarray(opath)
            da = standardize(da)
        else:
            da = _compute(base_da.sel(time=base_da.time.dt.year==year), progress=True).resample(time="1D").mean()
        da.to_netcdf(opath)

# Climatologies, datahandlers of new data

In [ ]:
compute_all_smoothed_anomalies("ERA5", "surf", "t2m", "dailymean", 'dayofyear', {'dayofyear': ('win', 15)}, None)

In [ ]:
compute_all_smoothed_anomalies("ERA5", "surf", "tp", "dailymean", 'dayofyear', {'dayofyear': ('win', 15)}, None)

In [ ]:
compute_all_smoothed_anomalies("ERA5", "surf", "sst", "dailymean", 'dayofyear', {'dayofyear': ('win', 15)}, None)

In [ ]:
compute_all_smoothed_anomalies("ERA5", "surf", "mslp", "dailymean", 'dayofyear', {'dayofyear': ('win', 15)}, None)

In [2]:
compute_all_smoothed_anomalies("ERA5", "thetalev", "apvs", "dailymean", 'dayofyear', {'dayofyear': ('win', 15)}, None)

In [ ]:
compute_all_smoothed_anomalies("ERA5", "thetalev", "cpvs", "dailymean", 'dayofyear', {'dayofyear': ('win', 15)}, None)

In [ ]:
for year in tqdm(YEARS):
    da = open_da("ERA5", "thetalev", "apvs", "6H", period=[year])
    da = compute(da.chunk({"lev": 1}).resample(time="1d").mean(), progress_flag=True)
    da.to_netcdf(data_path("ERA5", "thetalev", "apvs", "dailymean").joinpath(f"{year}.nc"))

In [ ]:
for year in tqdm(YEARS):
    da = open_da("ERA5", "plev", "z", "6H", period=[year])
    da = compute(da.chunk({"lev": 1}).resample(time="1d").mean(), progress_flag=True)
    da.to_netcdf(data_path("ERA5", "plev", "z", "dailymean").joinpath(f"{year}.nc"))

In [5]:
compute_all_smoothed_anomalies("ERA5", "plev", "z", "dailymean", 'dayofyear', {'dayofyear': ('win', 15)}, None)

  0%|          | 0/3 [00:00<?, ?it/s]

[########################################] | 100.00% Completed | 38.54 s


 33%|███▎      | 1/3 [00:39<01:18, 39.02s/it]

[########################################] | 100.00% Completed | 19.50 s


 67%|██████▋   | 2/3 [00:58<00:27, 27.71s/it]

[########################################] | 100.00% Completed | 20.58 s


100%|██████████| 64/64 [00:28<00:00,  2.23it/s]


In [5]:
for year in tqdm(YEARS):
    da = open_da("ERA5", "thetalev", "cpvs", "6H", period=[year])
    da = compute(da.chunk({"lev": 1}).resample(time="1d").mean(), progress_flag=False)
    da.to_netcdf(data_path("ERA5", "thetalev", "cpvs", "dailymean").joinpath(f"{year}.nc"))

100%|██████████| 64/64 [20:55<00:00, 19.61s/it]


In [4]:
da_ = compute(da.sel(lev=350), progress_flag=True)

[######                                  ] | 17.19% Completed | 34.36 ss


KeyboardInterrupt: 

In [1]:
da_

NameError: name 'da_' is not defined

In [5]:
da_ = pl.from_pandas(da_.to_dataframe().reset_index()).cast({"lat": pl.Float32, "lon": pl.Float32, "lev": pl.UInt16})

: 

: 

: 

In [ ]:
da_

In [13]:
da_.group_by(pl.col("time").dt.ordinal_day(), pl.col("lat"), pl.col("lon")).agg(pl.col("apvs").mean())

time,lat,lon,lev,apvs
datetime[ns],f32,f32,u16,f32
1960-01-01 00:00:00,0.0,-180.0,350,0.0
1960-01-01 00:00:00,0.0,-179.5,350,0.0
1960-01-01 00:00:00,0.0,-179.0,350,0.0
1960-01-01 00:00:00,0.0,-178.5,350,0.0
1960-01-01 00:00:00,0.0,-178.0,350,0.0
…,…,…,…,…
1960-12-31 18:00:00,90.0,177.5,350,0.0
1960-12-31 18:00:00,90.0,178.0,350,0.0
1960-12-31 18:00:00,90.0,178.5,350,0.0


In [10]:
compute_all_smoothed_anomalies("ERA5", "thetalev", "apvs", "6H", 'hourofyear', {'hourofyear': ('win', 60)}, None)

Slicing is producing a large chunk. To accept the large
chunk and silence this warning, set the option
    >>> with dask.config.set(**{'array.slicing.split_large_chunks': False}):
    ...     array[indexer]

To avoid creating the large chunks, set the option
    >>> with dask.config.set(**{'array.slicing.split_large_chunks': True}):
    ...     array[indexer]
  0%|          | 0/11 [00:00<?, ?it/s]2024-10-30 11:30:21,960 - distributed.worker - ERROR - Worker stream died during communication: tcp://127.0.0.1:33771
Traceback (most recent call last):
  File "/storage/homefs/hb22g102/miniforge3/envs/env11_2/lib/python3.11/site-packages/distributed/comm/tcp.py", line 225, in read
    frames_nosplit_nbytes_bin = await stream.read_bytes(fmt_size)
                                ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
tornado.iostream.StreamClosedError: Stream is closed

The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  File "/storage/homefs/hb

2024-10-30 11:30:21,507 - distributed.nanny.memory - WARNING - Worker tcp://127.0.0.1:33771 (pid=1471376) exceeded 95% memory budget. Restarting...
2024-10-30 11:30:22,042 - distributed.scheduler - WARNING - Removing worker 'tcp://127.0.0.1:33771' caused the cluster to lose already computed task(s), which will be recomputed elsewhere: {('array-c425aede2f8327a76e3f05f022d67563', 31), ('array-c425aede2f8327a76e3f05f022d67563', 21), ('array-c425aede2f8327a76e3f05f022d67563', 30), ('array-c425aede2f8327a76e3f05f022d67563', 20), ('array-c425aede2f8327a76e3f05f022d67563', 17), ('array-c425aede2f8327a76e3f05f022d67563', 16), ('reindex_intermediates-354bdf9a4a7033ddf7ff979cd099de20', 0, 0, 11), 'original-open_dataset-dummy-6948b1c41422ef3c78662e232b699ee1', ('reindex_intermediates-354bdf9a4a7033ddf7ff979cd099de20', 0, 0, 4)} (stimulus_id='handle-worker-cleanup-1730284222.0420094')
2024-10-30 11:30:22,225 - distributed.nanny - WARNING - Restarting worker
2024-10-30 11:30:26,116 - distributed.na

KeyboardInterrupt: 

2024-10-30 11:31:05,360 - distributed.worker.state_machine - WARNING - Async instruction for <Task cancelled name="execute(('groupby_nanmean-reduce-partial-57035dec009cfa2851df8bc7d4ecf2d7', 0, 0, 15))" coro=<Worker.execute() done, defined at /storage/homefs/hb22g102/miniforge3/envs/env11_2/lib/python3.11/site-packages/distributed/worker_state_machine.py:3615>> ended with CancelledError
2024-10-30 11:31:05,362 - distributed.worker - ERROR - Failed to communicate with scheduler during heartbeat.
Traceback (most recent call last):
  File "/storage/homefs/hb22g102/miniforge3/envs/env11_2/lib/python3.11/site-packages/distributed/comm/tcp.py", line 225, in read
    frames_nosplit_nbytes_bin = await stream.read_bytes(fmt_size)
                                ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
tornado.iostream.StreamClosedError: Stream is closed

The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  File "/storage/homefs/hb22g102/miniforge3/

In [ ]:
compute_all_smoothed_anomalies("ERA5", "thetalev", "cpvs", "6H", 'hourofyear', {'hourofyear': ('win', 60)}, None)

# new pvs das

In [2]:
import polars_st as st

ERROR 1: PROJ: proj_create_from_database: Open of /storage/homefs/hb22g102/miniforge3/envs/env11_2/share/proj failed


In [3]:
def to_xarray(events: st.GeoDataFrame, dummy_da: xr.DataArray, varname: str):
    dummy_da = dummy_da.rename("dummy")
    da_df = pl.from_pandas(dummy_da.to_dataframe().reset_index())
    orig_times = da_df["time"]
    timedtype = orig_times.dtype
    da_df = da_df.drop("time", "dummy")
    da_df = da_df.unique(["lat", "lon"]).sort(["lat", "lon"]).with_columns(geometry=st.from_xy("lon", "lat"))
    da_df = st.GeoDataFrame(da_df)
    events = events.with_columns(pl.col("geometry").st.buffer(0.25))
    events = (
        events.select(["date", "geometry", varname])
        .cast({varname: pl.Float32})
        .rename({"date": "time"})
    )
    events = events.cast({"time": timedtype})
    events = events.filter(pl.col("time").is_in(orig_times))
    dummy_da = xr.zeros_like(dummy_da, dtype=np.float32)
    events = events.st.sjoin(da_df, on="geometry", how="inner", predicate="contains")
    events = events.unique(["time", "lon", "lat"])
    events_da = xr.DataArray.from_series(
        events[["time", varname, "lat", "lon"]]
        .to_pandas()
        .set_index(["time", "lat", "lon"])[varname]
    ).fillna(0)
    dummy_da.loc[
        {
            "time": events_da.time.values,
            "lat": events_da.lat.values,
            "lon": events_da.lon.values,
        }
    ] = events_da
    return dummy_da

In [4]:
all_events = {}
for level in trange(310, 365, 5):
    events = st.from_geopandas(gpd.read_parquet(f"/storage/workspaces/giub_meteo_impacts/ci01/ERA5/RWB_index/era5_pv_streamers_{level}K_1959-2022.parquet"))

    tropospheric = events.filter(pl.col("mean_var") < pl.col("level"))
    anticyclonic = tropospheric.filter(pl.col("intensity") >= pl.col("level"))
    cyclonic = tropospheric.filter(pl.col("intensity") < pl.col("level"))
    
    all_events[level] = {"anti": anticyclonic, "cycl": cyclonic}

100%|██████████| 11/11 [00:47<00:00,  4.30s/it]


In [5]:
for year in YEARS:
    # for month in range(1, 13):
    ofile_anti = Path(f"{DATADIR}/ERA5/thetalev/apvs/6H/{year}.nc")
    ofile_cycl = Path(f"{DATADIR}/ERA5/thetalev/cpvs/6H/{year}.nc")
    if ofile_cycl.is_file() and ofile_anti.is_file():
        continue
    time_mask = (TIMERANGE.year == year)# & (TIMERANGE.month == month)
    coords = {
        "time": TIMERANGE[time_mask],
        "lat": np.arange(0, 90.5, .5),
        "lon": np.arange(-180, 180, .5),
    }
    shape = [len(co) for co in coords.values()]
    dummy_da = xr.DataArray(np.zeros(shape), coords=coords)
    anti_all_levs = {}
    cycl_all_levs = {}
    for lev, events in tqdm(all_events.items(), total=11):
        anti_all_levs[lev] = to_xarray(events["anti"], dummy_da, "intensity")
        cycl_all_levs[lev] = to_xarray(events["cycl"], dummy_da, "intensity")
    anti_all_levs = xr.concat(anti_all_levs.values(), dim="lev").assign_coords(lev=list(anti_all_levs))
    cycl_all_levs = xr.concat(cycl_all_levs.values(), dim="lev").assign_coords(lev=list(cycl_all_levs))
    anti_all_levs.to_netcdf(ofile_anti)
    cycl_all_levs.to_netcdf(ofile_cycl)

100%|██████████| 11/11 [08:44<00:00, 47.69s/it]


# CESM

In [1]:
from pathlib import Path
import numpy as np
import xarray as xr
basepath = Path("/storage/workspaces/giub_meteo_impacts/ci01/CESM2/flat_wind")
paths = list(basepath.iterdir())
paths = [path for path in paths if path.suffix == ".nc" and path.name != "ds.nc"]
parts = [path.name.split(".")[0].split("-") for path in paths]
parts = np.asarray(parts)
sorted_order = np.argsort([memb.replace("r10", "r0") for memb in parts[:, 0]])
parts = parts[sorted_order]
paths = [paths[i] for i in sorted_order]
all_members = np.unique(parts[:, 0])
all_years = np.unique(parts[:, 1])

not_here = []
here = []
for year in all_years:
    for member in all_members:
        potential_path = basepath.joinpath(f"{member}-{year}.nc")
        if potential_path.is_file():
            here.append(potential_path)
        else:
            not_here.append(potential_path)
len(here)

from itertools import groupby
paths_to_load = []
valid_ensembles = []
for key, indices in groupby(range(len(parts)), lambda i: parts[i][0]):
    indices = list(indices)
    group = parts[indices]
    these_paths = [paths[i] for i in indices]
    years = np.asarray(list([g[1] for g in group]))
    if len(years) == 60:
        paths_to_load.append(these_paths)
        valid_ensembles.append(key)
    else:
        print(key, len(years))

In [11]:
from tqdm import tqdm
ds = []
for ptl in tqdm(paths_to_load):
    ds_ = []
    for p in ptl:
        this = xr.open_dataset(p)
        this = this.reset_coords("time_bnds", drop=True).drop_dims("nbnd")
        ds_.append(this)
    ds.append(xr.concat(ds_, dim="time"))
ds = xr.concat(ds, dim="member")
# ds = xr.concat([xr.concat([xr.open_dataset[ptl_] for ptl_ in ptl], dim="time") for ptl in paths_to_load], dim="member")

In [ ]:
import dask
from dask.distributed import progress, Client
from jetstream_hugo.definitions import COMPUTE_KWARGS
client = Client(**COMPUTE_KWARGS)
dask.persist(ds)
progress(ds, notebook=False)
ds = dask.compute(ds)

In [16]:
ds = ds[0]
to_comp = ds.to_zarr(f"/storage/workspaces/giub_meteo_impacts/ci01/CESM2/flat_wind/ds.zarr", compute=False, encoding={var: {"chunks": (-1, 100, -1, -1)} for var in ds.data_vars}, mode="w")
dask.persist(to_comp)
progress(to_comp, notebook=False)
dask.compute(to_comp)

# extreme cesm clim

In [16]:
import numpy as np
from jetstream_hugo.data import *
quantiles = ds["s"].quantile(np.arange(0.6, 1, 0.05), ["member", "lon", "lat"]).compute()
quantiles = smooth(quantiles, {"time": ("win", 15)}).load()
quantiles.to_netcdf(f"{DATADIR}/CESM2/flat_wind/results/s_q.nc")


In [19]:
from matplotlib.dates import DateFormatter, MonthLocator
from jetstream_hugo.definitions import *
from scipy.stats import linregress

def get_trend(da):
    years = np.unique(da.time.dt.year)
    if "jet" not in da.dims:
        result = linregress(years, da.values)
        return xr.Dataset({"slope": result.slope, "p": result.pvalue})
    jets = da.jet.values
    slopes = xr.DataArray(np.zeros(len(jets)), coords={"jet": jets})
    pvalues = slopes.copy()
    for j, jet in enumerate(jets):
        result = linregress(years, da.isel(jet=j).values)
        slopes[j] = result.slope
        pvalues[j] = result.pvalue
    return xr.Dataset({"slope": slopes, "p": pvalues})

winsize = 15
halfwinsize = int(np.ceil(winsize / 2))

fig, ax = plt.subplots()
for q, qval in zip(quantiles[::2], np.arange(0.6, 1, 0.05 * 2)):
    gb = q.groupby("time.dayofyear")
    x = list(gb.groups)
    x = DATERANGE[x]
    ys = gb.map(get_trend) 
    ps = ys["p"]
    ys = ys["slope"]
    ys = ys.pad({"dayofyear": halfwinsize}, mode="wrap")
    ys = ys.rolling(dayofyear=winsize, center=True).mean()
    ys = ys.isel({"dayofyear": slice(halfwinsize, -halfwinsize)})
    ax.plot(x, ys, label=f"$q={qval:.1f}$", lw=2)
ax.grid(True)
ax.xaxis.set_major_locator(MonthLocator(range(0, 13, 3)))
ax.xaxis.set_major_formatter(DateFormatter("1 %b"))
ax.set_xlim(min(x), max(x))
ax.legend()

In [10]:
q_clim = compute_clim(quantiles, "dayofyear")
q_clim = smooth(q_clim, {"dayofyear": ("win", 61)}).load()
q_clim.to_netcdf(f"{DATADIR}/CESM2/flat_wind/results/s_q_clim.nc")

In [19]:
ds = ds.reset_coords("time_bnds", drop=True)

# Extreme experiment

In [ ]:
exp_tp = ExtremeExperiment(
    DataHandler("ERA5", "surf", "tp", "6H", "all", "JJA", -30, 40, 30, 75, 250, 'hourofyear', {'hourofyear': ('win', 4 * 15)}, None),
    q = 0.95,
)
da_tp = exp_tp.da.load()

data_handlers = {}
for varname in ["u", "v", "s"]:
    dh = DataHandler("ERA5", "plev", varname, "6H", "all", None, -80, 40, 15, 80, [175, 200, 225, 250, 300, 350], reduce_da=False)
    data_handlers[varname] = dh
exp = MultiVarExperiment(data_handlers)

all_jets_one_df, where_are_jets, all_jets_one_array, all_jets_over_time, flags = exp.track_jets()
props_as_ds = exp.props_as_ds(True)

In [ ]:
from deepdiff import DeepHash
DeepHash(load_pickle("/storage/workspaces/giub_meteo_impacts/ci01/ERA5/surf/tp/6H/hourofyear_hourofyearwin60/results/1/predictions/1/metadata.pkl"))

In [ ]:
subset = ["mean_lon", "mean_lat", "mean_lev", "spe_star", "width", "wavinessR16", "persistence", "com_speed", "int"]
predictors = prepare_predictors(
    props_as_ds,
    subset=subset,
    anomalize=True,
    normalize=True,
    detrend=True,
    nan_method="nearest",
    season="JJA",
)
time_before = pd.Timedelta(0, "D")
n_clu = 22
clusters_da = exp_tp.spatial_clusters_as_da(n_clu)
targets, length_targets, all_spells_ts, all_spells = exp_tp.create_targets(n_clu, 0.95, minlen=np.timedelta64(1, "D"))
binary_targets = length_targets > 0
masked_predictors = mask_from_spells_multi_region(predictors, targets, all_spells_ts, all_spells, time_before=time_before)

In [ ]:
clu = Clusterplot(1, 1, exp_tp.region)
cmap = colormaps.BlAqGrYeOrReVi200
ax = clu.axes[0]
unique_clusters = np.arange(n_clu)
norm = BoundaryNorm(np.concatenate([[-1], unique_clusters]) + 0.5, cmap.N)
clusters_da.unstack().plot(
    ax=ax,
    cmap=cmap,
    norm=norm,
    add_colorbar=False,
    add_labels=False
)
for j in range(n_clu):
    lo = clusters_da.lon.where(clusters_da==j).mean().item()
    la = clusters_da.lat.where(clusters_da==j).mean().item()
    ax.text(lo, la, f"${j}$", ha="center", va="center", fontweight="bold")

In [ ]:
compute_all_smoothed_anomalies("ERA5", "plev", "s", "6H", 'hourofyear', {'hourofyear': ('win', 4 * 15)}, None)

In [ ]:
compute_all_smoothed_anomalies("ERA5", "surf", "tp", "6H", 'hourofyear', {'hourofyear': ('win', 4 * 15)}, None)

In [ ]:
basepath = Path(f"{DATADIR}/ERA5/surf")
varnames = ["u10", "v10", "s10"]
for year, month in tqdm(product(YEARS, range(1, 13)), total=len(YEARS) * 12):
    month_str = str(month).zfill(2)
    ofiles = {varname: basepath.joinpath(f"{varname}/6H/{year}{month_str}.nc") for varname in varnames}
    if all([ofile.is_file() for ofile in ofiles.values()]):
        continue
    ds = xr.open_dataset(basepath.joinpath(f"raw/{year}{month_str}.nc"))
    ds = ds.rename(longitude="lon", latitude="lat")
    ds = ds.assign_coords(lon=(((ds.lon + 180) % 360) - 180))
    ds = ds.sortby("lon")
    ds = ds.sortby("lat")
    ds["s10"] = np.sqrt(ds["u10"] ** 2 + ds["v10"] ** 2)
    for varname in varnames:
        da = ds[varname]
        with warnings.catch_warnings():
            warnings.simplefilter("ignore", category=xr.SerializationWarning)
            da.to_netcdf(ofiles[varname])